In [3]:
# Change directory so that code in "src" directory is easily importable.
import os
os.chdir('../')

In [4]:
import pandas as pd
import numpy as np
import re

from pyathena import connect
from pyathena.pandas_cursor import PandasCursor

from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

from mord import LogisticAT

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.metrics import make_scorer, SCORERS, f1_score, precision_score, recall_score

import tensorflow as tf
from tensorflow import keras

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Define connection to DB
conn = connect(
    s3_staging_dir='s3://aws-athena-query-results-323906537337-us-east-1/',
    region_name='us-east-1',
    cursor_class=PandasCursor
    )
cursor = conn.cursor()

# 0 Data

## 0.1 get player data

In [6]:
simple_query = f'''
SELECT
    CAST(season AS INTEGER) season,
    eventmetadata.gamecode game_code,
    player.playerid player_id, team.teamid team_id, positionid position_id, 
    isgameplayed game_played, 
    isgamestarted game_started,
    minutesplayed minutes,
    points,
    fieldgoals.attempted fg_attempt, 
    fieldgoals.made fg_made,
    freethrows.attempted ft_attempt, 
    freethrows.made ft_made,
    threepointfieldgoals.attempted point_3_attempt, 
    threepointfieldgoals.made point_3_made,
    rebounds.offensive offensive_rebounds, 
    rebounds.defensive defensive_rebounds, 
    assists, 
    blockedshots blocks, 
    turnovers
FROM 
    datalakebasketball.player_stats_game
WHERE 
    leagueid='1' 
    and eventmetadata.eventtypeid=1 
    and season>='2004'
ORDER BY 
    season, player_id, eventmetadata.gamedateutcepoch
'''

if True:
    player_df = cursor.execute(simple_query).as_pandas()
    print(player_df.info())
else:
    print("Failed to query!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568590 entries, 0 to 568589
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   season              568590 non-null  Int64
 1   game_code           568590 non-null  Int64
 2   player_id           568590 non-null  Int64
 3   team_id             568590 non-null  Int64
 4   position_id         234360 non-null  Int64
 5   game_played         568590 non-null  bool 
 6   game_started        568590 non-null  bool 
 7   minutes             568590 non-null  Int64
 8   points              568590 non-null  Int64
 9   fg_attempt          568590 non-null  Int64
 10  fg_made             568590 non-null  Int64
 11  ft_attempt          568590 non-null  Int64
 12  ft_made             568590 non-null  Int64
 13  point_3_attempt     568590 non-null  Int64
 14  point_3_made        568590 non-null  Int64
 15  offensive_rebounds  568590 non-null  Int64
 16  defensive_rebounds  